In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps

# Census & gmaps API Keys
from config import (api_key, gkey)
c = Census(api_key, year=2013)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

census_pd.head()

,State,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,01,Alabama,4799277.0,38.1,43253.0,23680.0,870631.0,18.140878,5.040968
1,02,Alaska,720316.0,33.6,70760.0,32651.0,69514.0,9.650487,4.572854
2,04,Arizona,6479703.0,36.3,49774.0,25358.0,1131901.0,17.468409,4.882323
3,05,Arkansas,2933369.0,37.5,40768.0,22170.0,547328.0,18.658682,4.132961
4,06,California,37659181.0,35.4,61094.0,29527.0,5885417.0,15.628107,5.758662


In [3]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data_states.csv", encoding="utf-8", index=False)

In [4]:
# Read in the csv containing state centroid coordinates
centroids = pd.read_csv("../Resources/state_centroids.csv")
centroids.head()

,State,Latitude,Longitude
0,Alabama,32.7794,-86.8287
1,Alaska,64.0685,-152.2782
2,Arizona,34.2744,-111.6602
3,Arkansas,34.8938,-92.4426
4,California,37.1841,-119.4696


In [5]:
# Merge the datasets using the sate columns
census_data = pd.merge(census_pd, centroids, how="left", left_on="Name", right_on="State")

# Save the updated dataframe as a csv
census_data.to_csv("../Resources/state_census_data.csv", encoding="utf-8", index=False)
census_data.head()

,State_x,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,State_y,Latitude,Longitude
0,01,Alabama,4799277.0,38.1,43253.0,23680.0,870631.0,18.140878,5.040968,Alabama,32.7794,-86.8287
1,02,Alaska,720316.0,33.6,70760.0,32651.0,69514.0,9.650487,4.572854,Alaska,64.0685,-152.2782
2,04,Arizona,6479703.0,36.3,49774.0,25358.0,1131901.0,17.468409,4.882323,Arizona,34.2744,-111.6602
3,05,Arkansas,2933369.0,37.5,40768.0,22170.0,547328.0,18.658682,4.132961,Arkansas,34.8938,-92.4426
4,06,California,37659181.0,35.4,61094.0,29527.0,5885417.0,15.628107,5.758662,California,37.1841,-119.4696


In [7]:
# Convert poverty rate as a list
# Convert bank rate to list
poverty_rate = census_data["Poverty Rate"].tolist()

In [ ]:
# Create a map using state centroid coordinates to set markers
marker_locations = census_data[['Latitude', 'Longitude']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
fig.add_layer(markers)
fig

![10-State_Markers.png](../Images/10-State_Markers.png)